In [1]:
from cgmbrush import *
from tempfile import TemporaryFile
import cProfile
import io
import pstats
from scipy.ndimage.filters import convolve
import scipy.signal as sig


32512

In [ ]:
# Perf testing for reading Bolshoi density fields

provider = BolshoiProvider()

pr = cProfile.Profile()
pr.enable()
den_field = provider.get_density_field(0, 256)
pr.disable()
s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('tottime')
ps.print_stats()

file_path = os.path.join(varFolder, 'bolshoi_density_perf.txt')
with open(file_path, 'w') as f:
    f.write(s.getvalue())

saveArray("perf_denfield", den_field)

In [ ]:
provider = BolshoiProvider()

pr = cProfile.Profile()
pr.enable()
read_denfield = loadArray("perf_denfield.npy")
pr.disable()
s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('tottime')
ps.print_stats()

file_path = os.path.join(varFolder, 'bolshoi_density_perf_read.txt')
with open(file_path, 'w') as f:
    f.write(s.getvalue())

In [ ]:
# Testing savez
outfile = TemporaryFile()
x = np.arange(1000)
y = np.arange(1000) + 1

arrays = {'y':y, 'x':x, }

np.savez(outfile, **arrays)
_ = outfile.seek(0)
npzfile = np.load(outfile)
npzfile['x']

In [ ]:
p = BolshoiProvider()
a = p.extract_halos(0)
print(a.shape)
b = loadArray('bol_halos_0.npy')
print(b.shape)

In [ ]:
print(a.shape)
print(a.info(memory_usage='deep'))
print(b.shape)
print(type(b))
print(str((b.size * b.itemsize)/1024**2) + " MB")
print(b[0])

In [2]:
# Profiling full convolution code for simplest case

min_mass = 10**10 # TODO into config
max_mass = 10**14.5
log_bins = 30
config = Configuration('tophat_spherical', 1, resolution=4, file_prefix='STH', load_from_files=False)
config.provider = BolshoiProvider()
filename = config.file_prefix + str(config.resolution) + '_' + str(config.den_grid_size)


pr = cProfile.Profile()
pr.enable()
hist_profile(config.provider, config.den_grid_size, config.RS_array, min_mass, 
                                    max_mass, log_bins, config.subtraction_halo_profile, 
                                    config.addition_halo_profile, config.scaling_radius, config.resolution)
pr.disable()
s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('tottime')
ps.print_stats()

file_path = os.path.join(varFolder, 'perf_convo_' + filename + '_v3.txt')
with open(file_path, 'w') as f:
    f.write(s.getvalue())

#print(config.results)
#saveArray(filename, config.results)
     




/Users/ianw89/Repos/cgm-brush/src/cgmbrush.py:456: RuntimeWarning: invalid value encountered in double_scalars
  return integrate.quad(lambda x, y, z: rho_nought/(((offset+(x**2+y**2+ z**2)**.5)/R_s)*(1+((x**2+y**2+ z**2)**.5)/R_s)**2),-1*np.real((Rvir**2-(x**2+y**2))**.5),np.real((Rvir**2-(x**2+y**2))**.5),args=(x,y))[0]
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/scipy/integrate/quadpack.py:385: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)


Starting async tasks at 1629139327.78631
PID 5918: adding halos for bin 0
PID 5919: adding halos for bin 1
PID 5920: adding halos for bin 2
PID 5921: adding halos for bin 3
PID 5922: adding halos for bin 4
PID 5923: adding halos for bin 5
PID 5924: adding halos for bin 6
PID 5925: adding halos for bin 7
PID 5926: adding halos for bin 8
PID 5927: adding halos for bin 9
PID 5928: adding halos for bin 10
PID 5929: adding halos for bin 11
PID 5930: adding halos for bin 12
PID 5931: adding halos for bin 13
PID 5932: adding halos for bin 14
PID 5933: adding halos for bin 15
PID 5934: adding halos for bin 16
PID 5935: adding halos for bin 17
PID 5936: adding halos for bin 18
PID 5937: adding halos for bin 19
PID 5938: adding halos for bin 20
PID 5939: adding halos for bin 21
PID 5940: adding halos for bin 22
PID 5941: adding halos for bin 23
PID 5942: adding halos for bin 24
PID 5943: adding halos for bin 25
PID 5944: adding halos for bin 26
PID 5945: adding halos for bin 27
PID 5946: adding 

In [ ]:
# Profiling full convolution code for simplest case
min_mass = 10**10 # TODO into config
max_mass = 10**14.5
log_bins = 30
config = Configuration('tophat_spherical', 1, resolution=16, file_prefix='STH', load_from_files=False)
config.provider = BolshoiProvider()
filename = config.file_prefix + str(config.resolution) + '_' + str(config.den_grid_size)


pr = cProfile.Profile()
pr.enable()
config.results = hist_profile(config.provider, config.den_grid_size, config.RS_array, min_mass, 
                                    max_mass, log_bins, config.subtraction_halo_profile, 
                                    config.addition_halo_profile, config.scaling_radius, config.resolution)
pr.disable()
s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('tottime')
ps.print_stats()

saveArray(filename, config.results)

file_path = os.path.join(varFolder, 'perf_convo_' + filename + '_v2.txt')
with open(file_path, 'w') as f:
    f.write(s.getvalue())

SCRATCH WORK BELOW HERE

In [ ]:
a = np.random.random((1024,1024))
b = np.random.random((20,20))

In [ ]:
# This is the convolve function we are using today, but in a mode where the big array is given 0's outside the edge of the grid
# This is 'wrong' for our application but it makes for an easy comparison with the next function, which doesn't support 
# the reflect mode we use.
c1 = convolve(a, b, mode='constant', origin=-1)

In [ ]:
# The scipy signal implementation is much faster. But it only supports 0's around the edge
c2 = sig.convolve(a, b, mode='same')
np.allclose(c1,c2)

In [ ]:
# This is almost what we do today: just same method but using wrap instead of reflect
c7 = convolve(a, b, mode='wrap', origin=-1) # a b c d | a b c d | a b c d
c8 = convolve(a, b, mode='wrap')

In [ ]:
c3 = my_convolve(a, b)
print(np.allclose(c7,c3))
print(np.allclose(c8,c3)) # My implementation is equivalent to what we did before!


In [ ]:
# This is what we are actually running today. The same code as c1 but in reflect mode. d c b a | a b c d | d c b a
c4 = convolve(a, b, origin=-1)
print(np.allclose(c4,c3))
print(np.allclose(c4,c1))     

In [ ]:
# Testing signal's convolve 2d. It's too slow.
c5 = sig.convolve2d(a, b, mode='same', boundary='symm')
print(np.allclose(c5,c1))
print(np.allclose(c5,c2))
print(np.allclose(c5,c3))
print(np.allclose(c5,c4)) # interestingly it does match what we dotoday exactly

In [ ]:
# Valid mode basically trims the edges. This is proof that using signal in this is equivalent to what we are doing today,
# but throwing away a tiny bit of data!
c6 = sig.convolve(a, b, mode='valid')
print(np.allclose(c1[40:-39, 40:-39], c6))
print(np.allclose(c4[40:-39, 40:-39], c6))
print(np.allclose(c4[39:-40, 39:-40], c6))